<img src="https://8weeksqlchallenge.com/images/case-study-designs/1.png" width="350">

Thank you to Danny Ma for creating the following case study.

## **Introduction**

See more information on the scenario [here](https://8weeksqlchallenge.com/case-study-1/?utm_source=ActiveCampaign&utm_medium=email&utm_content=6%20unique%20SQL%20resources%20you%20haven%20t%20seen%20before%20).

Danny seriously loves Japanese food. So, in the beginning of 2021, he decided to embark upon a risky venture and opens up a cute little restaurant that sells his 3 favourite foods: sushi, curry and ramen.

Danny’s Diner is in need of your assistance to help the restaurant stay afloat - the restaurant has captured some very basic data from their few months of operation but have no idea how to use their data to help them run the business.

### **Datasets**

All datasets exist within the `dannys_diner` database schema

#### Table 1: `sales`

The sales table captures all `customer_id` level purchases with an corresponding `order_date` and `product_id` information for when and what menu items were ordered.

| customer_id | order_date | product_id |
| ----------- | ---------- | ---------- |
| A           | 2021-01-01 | 1          |
| A           | 2021-01-01 | 2          |
| A           | 2021-01-07 | 2          |
| A           | 2021-01-10 | 3          |
| A           | 2021-01-11 | 3          |
| A           | 2021-01-11 | 3          |
| B           | 2021-01-01 | 2          |
| B           | 2021-01-02 | 2          |
| B           | 2021-01-04 | 1          |
| B           | 2021-01-11 | 1          |
| B           | 2021-01-16 | 3          |
| B           | 2021-02-01 | 3          |
| C           | 2021-01-01 | 3          |
| C           | 2021-01-01 | 3          |
| C           | 2021-01-07 | 3          |

---

[View on DB Fiddle](https://www.db-fiddle.com/f/2rM8RAnq7h5LLDTzZiRWcd/138)

#### Table 2: `menu`

The `menu` table maps the `product_id` to the actual `product_name` and `price` of each menu item.

| product_id | product_name | price |
| ---------- | ------------ | ----- |
| 1          | sushi        | 10    |
| 2          | curry        | 15    |
| 3          | ramen        | 12    |

---

[View on DB Fiddle](https://www.db-fiddle.com/f/2rM8RAnq7h5LLDTzZiRWcd/138)

#### Table 3: `members`

The `members` table captures the `join_date` when a `customer_id` joined the beta version of the Danny’s Diner loyalty program.

| customer_id | join_date  |
| ----------- | ---------- |
| A           | 2021-01-07 |
| B           | 2021-01-09 |

---

[View on DB Fiddle](https://www.db-fiddle.com/f/2rM8RAnq7h5LLDTzZiRWcd/138)

## **Questions**

####1. What is the total amount each customer spent at the restaurant?

Customer A spent the most at the restaurant with \$76. Customer B spent \$74, and customer C spent \$36.


| customer_id | total_spent |
| ----------- | ----------- |
| A           | 76          |
| B           | 74          |
| C           | 36          |

---

[View on DB Fiddle](https://www.db-fiddle.com/f/2rM8RAnq7h5LLDTzZiRWcd/138)


    SELECT
      s.customer_id,
      SUM(m.price) AS total_spent
    FROM
      sales AS s
      LEFT JOIN
        menu AS m
          ON
            s.product_id = m.product_id
    GROUP BY
      s.customer_id
    ORDER BY
      s.customer_id
    ;

####2. How many days has each customer visited the restaurant?

Customer B has visited the restaurant six times, the most. Customer A has visited four times, and customer C has visited twice.


| customer_id | day_count |
| ----------- | --------- |
| A           | 4         |
| B           | 6         |
| C           | 2         |

---

[View on DB Fiddle](https://www.db-fiddle.com/f/2rM8RAnq7h5LLDTzZiRWcd/138)

    SELECT
      customer_id,
      COUNT(DISTINCT order_date) AS day_count
    FROM
      sales
    GROUP BY
      customer_id
    ORDER BY
      customer_id
    ;

**Note:** this query works under the (correct) assumption that `order_date` is only precise to the date, thus two purchases made on the same day will have identitcal values. If this column included the time of the purchase, we would have to change the query to not over count customers with multiple purchases in the same day. This step is redundant given the dataset so was left out of the official solution.

    SELECT
      customer_id,
      COUNT(DISTINCT TO_CHAR(order_date, 'MM-DD-YYYY')) AS day_count
    FROM
      sales
    GROUP BY
      customer_id
    ORDER BY
      customer_id
    ;

####3. What was the first item from the menu purchased by each customer?

Customer A purchased curry and sushi the first time they ordered. Customer B ordered curry, and customer C ordered ramen.


| customer_id | product_name |
| ----------- | ------------ |
| A           | curry        |
| A           | sushi        |
| B           | curry        |
| C           | ramen        |

---

[View on DB Fiddle](https://www.db-fiddle.com/f/2rM8RAnq7h5LLDTzZiRWcd/138)

    SELECT
      DISTINCT s.customer_id,
      m.product_name
    FROM
      sales AS s
        LEFT JOIN
          menu AS m
            ON
              s.product_id = m.product_id
    WHERE
      (s.customer_id, s.order_date) IN (
          SELECT
            customer_id,
            MIN(order_date)
          FROM
            sales
          GROUP BY
            customer_id
        )
    ORDER BY
      customer_id,
      product_name
    ;

**Note:** in the event tie, this query includes every product as its own row. That is, if a customer ordered multiple products on their first day at the restaurant, then each item would be included in its own row.

####4. What is the most purchased item on the menu and how many times was it purchased by all customers?

Ramen is the most purchased item on the menu, having been ordered eight times.

| product_name | product_count |
| ------------ | ------------- |
| ramen        | 8             |

---

[View on DB Fiddle](https://www.db-fiddle.com/f/2rM8RAnq7h5LLDTzZiRWcd/138)

    WITH
      with_product_counts AS (
        SELECT
          m.product_name,
          COUNT(m.product_name) AS product_count,
          RANK() OVER(
              ORDER BY COUNT(m.product_name) DESC
            ) AS product_rank
        FROM
          sales AS s
          LEFT JOIN
            menu AS m
              ON
                s.product_id = m.product_id
          GROUP BY
            m.product_name
          ORDER BY
            product_count DESC
      )
     
    SELECT
      product_name,
      product_count
    FROM
      with_product_counts
    WHERE
      product_rank = 1
    ;

**Note:** the given solution accounts for the possibility of a tie in the number of products sold. If you made the assumption that such tie did not exist, the following query would yield the same result in a more efficient manner.

	SELECT
		m.product_name,
		COUNT(m.product_name) AS product_count
    FROM
		sales AS s
			LEFT JOIN
				menu AS m
					ON
						s.product_id = m.product_id
    GROUP BY
		m.product_name
    ORDER BY
		product_count DESC
    LIMIT 1
    ;

####5. Which item was the most popular for each customer?

Customers A and C both have ordered ramen the most. Customer B has ordered all three menu items equal amounts.

| customer_id | product_name | product_count |
| ----------- | ------------ | ------------- |
| A           | ramen        | 3             |
| B           | ramen        | 2             |
| B           | curry        | 2             |
| B           | sushi        | 2             |
| C           | ramen        | 3             |

---

[View on DB Fiddle](https://www.db-fiddle.com/f/2rM8RAnq7h5LLDTzZiRWcd/138)

    WITH
      product_counts AS (
        SELECT
          customer_id,
            m.product_name,
            COUNT(m.product_name) AS product_count,
            RANK() OVER(
                PARTITION BY s.customer_id
                ORDER BY COUNT(m.product_name) DESC
              ) AS product_rank
        FROM
          sales AS s
          LEFT JOIN
            menu AS m
              ON
                s.product_id = m.product_id
        GROUP BY
          m.product_name,
          s.customer_id
        ORDER BY
          s.customer_id
      )
     
    SELECT
      customer_id,
      product_name,
      product_count
    FROM
      product_counts
    WHERE
      product_rank = 1
    ;


####6. Which item was purchased first by each customer after they became a member?

Customer A ordered curry first after they became a member. Customer B ordered sushi.

| customer_id | product_name |
| ----------- | ------------ |
| A           | curry        |
| B           | sushi        |

---

[View on DB Fiddle](https://www.db-fiddle.com/f/2rM8RAnq7h5LLDTzZiRWcd/138)

    WITH
      products_ordered_ranked AS (
          SELECT
            s.customer_id,
            m.product_name,
            RANK() OVER(
                PARTITION BY s.customer_id
                ORDER BY s.order_date
              ) AS date_rank
          FROM
            sales AS s
              INNER JOIN
                members AS c
                  ON
                    c.customer_id = s.customer_id AND
                    s.order_date >= c.join_date
              LEFT JOIN
                menu AS m
                  ON
                    s.product_id = m.product_id
    )
    
    SELECT
      customer_id,
      product_name
    FROM
      products_ordered_ranked
    WHERE
      date_rank = 1
    ;


**Note:** this query makes the assumption that if a customer becomes a member the same date that they order from the restaurant then they became a member first. The query could be easily tweaked if we wanted to change the assumption that the purchase was made before the customer became a member. In the primary query we would change:<br>
`s.order_date >= c.join_date`<br>
to:<br>
`s.order_date > c.join_date`<br>
which would give the following table.


| customer_id | product_name |
| ----------- | ------------ |
| A           | ramen        |
| B           | sushi        |

---

[View on DB Fiddle](https://www.db-fiddle.com/f/2rM8RAnq7h5LLDTzZiRWcd/138)

####7. Which item was purchased just before each customer became a member?

Customer A ordered sushi and curry in their las order before they became a member. Customer B ordered sushi.


| customer_id | product_name |
| ----------- | ------------ |
| A           | sushi        |
| A           | curry        |
| B           | sushi        |

---

[View on DB Fiddle](https://www.db-fiddle.com/f/2rM8RAnq7h5LLDTzZiRWcd/138)

    WITH
      products_ordered_ranked AS (
          SELECT
            s.customer_id,
            m.product_name,
            RANK() OVER(
                PARTITION BY s.customer_id
                ORDER BY s.order_date DESC
              ) AS date_rank
          FROM
            sales AS s
              INNER JOIN
                members AS c
                  ON
                    c.customer_id = s.customer_id AND
                    s.order_date < c.join_date
              LEFT JOIN
                menu AS m
                  ON
                    s.product_id = m.product_id
        )
    
    SELECT
      customer_id,
      product_name
    FROM
      products_ordered_ranked
    WHERE
      date_rank = 1
    ;

This query is nearly identical to the one in question #6. There are two differences.

1.  In the `RANK()` window function in the `products_ordered_ranked` CTE, `ORDER BY s.order_date` is descending. This ranks the latest date 1, rather than the earliest date.
2.   The CTE mentioned above has a `LEFT JOIN` condition for the order date to be less than the join date, rather than greater than or equal to.

**Note:** this query makes the assumption that if a customer becomes a member the same date that they order from the restaurant then they became a member first. The query could be easily tweaked if we wanted to change the assumption that the purchase was made before the customer became a member. In the primary query, we would change:<br>
`s.order_date < c.join_date`<br>
to:<br>
`s.order_date <= c.join_date`<br>
which would give the following table.


| customer_id | product_name |
| ----------- | ------------ |
| A           | curry        |
| B           | sushi        |

---

[View on DB Fiddle](https://www.db-fiddle.com/f/2rM8RAnq7h5LLDTzZiRWcd/138)

####8. What is the total items and amount spent for each member before they became a member?

Prior to becoming members, customer A bought two products and spent \$25, while customer B bought three things and spent \$40.


| customer_id | product_count | total_spent |
| ----------- | ------------- | ----------- |
| A           | 2             | 25          |
| B           | 3             | 40          |

---

[View on DB Fiddle](https://www.db-fiddle.com/f/2rM8RAnq7h5LLDTzZiRWcd/138)

    SELECT
      s.customer_id,
      COUNT(m.product_name) AS product_count,
      SUM(m.price) AS total_spent
    FROM
      sales AS s
        INNER JOIN
          members AS c
              ON
                c.customer_id = s.customer_id AND
                s.order_date < c.join_date
        LEFT JOIN
          menu AS m
            ON
              s.product_id = m.product_id
    GROUP BY
      s.customer_id
    ORDER BY
      s.customer_id
    ;

**Note:** this query makes the same assumptions that a customer signs up for membership before ordering if they do both actions on the same day.

####9. If each $1 spent equates to 10 points and sushi has a 2x points multiplier - how many points would each customer have?

Customer A earned 510 points, and customer B earned 440 points.

| customer_id | points |
| ----------- | ------ |
| A           | 510    |
| B           | 440    |

---

[View on DB Fiddle](https://www.db-fiddle.com/f/2rM8RAnq7h5LLDTzZiRWcd/138)

    SELECT
      s.customer_id,
      SUM(
        CASE
          WHEN m.product_id = 1
            THEN 20*price
          ELSE
            10*price
        END) AS points
    FROM
      sales AS s
        INNER JOIN
          members AS c
            ON
              c.customer_id = s.customer_id AND
              s.order_date >= c.join_date
        LEFT JOIN
          menu AS m
            ON
              s.product_id = m.product_id
    GROUP BY
      s.customer_id
    ORDER BY
      s.customer_id
    ;

**Note:** this query makes the same assumptions that a customer signs up for membership before ordering if they do both actions on the same day. Further, it is assumed that a customer does not start accruing points until they sign up to be a member.

####10. In the first week after a customer joins the program (including their join date) they earn 2x points on all items, not just sushi - how many points do customer A and B have at the end of January?

Customer A has accrued 1020 points, while customer B has 320 points.


| customer_id | points |
| ----------- | ------ |
| A           | 1020   |
| B           | 320    |

---

[View on DB Fiddle](https://www.db-fiddle.com/f/2rM8RAnq7h5LLDTzZiRWcd/138)

    SELECT
      s.customer_id,
      SUM(CASE
          WHEN (
            (EXTRACT(EPOCH FROM s.order_date) - EXTRACT(EPOCH FROM c.join_date))/86400) < 7
              OR m.product_id = 1
                THEN 20*price
          ELSE
            10*price
        END) AS points
    FROM
      sales AS s
        INNER JOIN
          members AS c
            ON
              c.customer_id = s.customer_id AND
              s.order_date >= c.join_date AND
              s.order_date < '2021-02-01'
        LEFT JOIN
          menu AS m
            ON
              s.product_id = m.product_id
    GROUP BY
      s.customer_id
    ORDER BY
      s.customer_id

###Bonus Questions
The following questions are related creating basic data tables that Danny and his team can use to quickly derive insights without needing to join the underlying tables using SQL.

####Bonus Question 1: Recreate the following table output using the available data.


| customer_id | order_date | product_name | price | member |
| ----------- | ---------- | ------------ | ----- | ------ |
| A           | 2021-01-01 | sushi        | 10    | N      |
| A           | 2021-01-01 | curry        | 15    | N      |
| A           | 2021-01-07 | curry        | 15    | Y      |
| A           | 2021-01-10 | ramen        | 12    | Y      |
| A           | 2021-01-11 | ramen        | 12    | Y      |
| A           | 2021-01-11 | ramen        | 12    | Y      |
| B           | 2021-01-01 | curry        | 15    | N      |
| B           | 2021-01-02 | curry        | 15    | N      |
| B           | 2021-01-04 | sushi        | 10    | N      |
| B           | 2021-01-11 | sushi        | 10    | Y      |
| B           | 2021-01-16 | ramen        | 12    | Y      |
| B           | 2021-02-01 | ramen        | 12    | Y      |
| C           | 2021-01-01 | ramen        | 12    | N      |
| C           | 2021-01-01 | ramen        | 12    | N      |
| C           | 2021-01-07 | ramen        | 12    | N      |

---

[View on DB Fiddle](https://www.db-fiddle.com/f/2rM8RAnq7h5LLDTzZiRWcd/138)

    SELECT
      s.customer_id,
      s.order_date,
      m.product_name,
      m.price,
        CASE
          WHEN c.join_date IS NULL
            THEN 'N'
          WHEN s.order_date < c.join_date
            THEN 'N'
          ELSE
            'Y'
          END AS member
    FROM
      sales AS s
        LEFT JOIN
          members AS c
            ON
              c.customer_id = s.customer_id
        LEFT JOIN
          menu AS m
            ON
              s.product_id = m.product_id
    ORDER BY
      s.customer_id,
      s.order_date
    ;

####Bonus Question 2: Danny also requires further information about the `ranking` of customer products, but he purposely does not need the ranking for non-member purchases so he expects null `ranking` values for the records when customers are not yet part of the loyalty program.


| customer_id | order_date | product_name | price | member | ranking |
| ----------- | ---------- | ------------ | ----- | ------ | ------- |
| A           | 2021-01-01 | sushi        | 10    | N      |         |
| A           | 2021-01-01 | curry        | 15    | N      |         |
| A           | 2021-01-07 | curry        | 15    | Y      | 1       |
| A           | 2021-01-10 | ramen        | 12    | Y      | 2       |
| A           | 2021-01-11 | ramen        | 12    | Y      | 3       |
| A           | 2021-01-11 | ramen        | 12    | Y      | 3       |
| B           | 2021-01-01 | curry        | 15    | N      |         |
| B           | 2021-01-02 | curry        | 15    | N      |         |
| B           | 2021-01-04 | sushi        | 10    | N      |         |
| B           | 2021-01-11 | sushi        | 10    | Y      | 1       |
| B           | 2021-01-16 | ramen        | 12    | Y      | 2       |
| B           | 2021-02-01 | ramen        | 12    | Y      | 3       |
| C           | 2021-01-01 | ramen        | 12    | N      |         |
| C           | 2021-01-01 | ramen        | 12    | N      |         |
| C           | 2021-01-07 | ramen        | 12    | N      |         |

---

[View on DB Fiddle](https://www.db-fiddle.com/f/2rM8RAnq7h5LLDTzZiRWcd/138)

    SELECT
      s.customer_id,
      s.order_date,
      m.product_name,
      m.price,
      'Y' AS member,
      RANK() OVER(
          PARTITION BY s.customer_id
          ORDER BY s.order_date
        ) AS ranking
    FROM
      sales AS s
        INNER JOIN
          members AS c
            ON
              c.customer_id = s.customer_id AND
              c.join_date <= s.order_date
        LEFT JOIN
          menu AS m
            ON
              s.product_id = m.product_id
          
          
    UNION ALL
        
    SELECT
      s.customer_id,
      s.order_date,
      m.product_name,
      m.price,
      'N' AS member,
      NULL AS ranking
    FROM
      sales AS s
        LEFT JOIN
          members AS c
            ON
              c.customer_id = s.customer_id AND
              c.join_date <= s.order_date
        LEFT JOIN
          menu AS m
            ON
              s.product_id = m.product_id
    WHERE
      c.join_date IS NULL
    ORDER BY
      1, 2
    ;
